# RecSeats

## Deep component : CNN and CDNN Models

Main file for running Convolutional Neural Network (CNN) and Convolutional/Deconvolutional Neural Network (CDNN) on Locational choice experiment data.

**Author of the code:** Anon.


#### Librairies import

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import copy

import csv
import numpy as np

import matplotlib.pyplot as plt

import os

#### Function from other files import

In [ ]:
from src.preprocessing.data_loading import load_data_matrix
from src.room_transformation import keep_left_seat
from src.deep.traintest import TrainTest
from src.utils import Params
from src.model.cnn import *
from src.model.autoencoder_cnn import *
from src.visualisation.plot_example import * 
from src.deep.deep_utils import *
from src.metrics import *

#### File setting

Changing the `IND_FILE` value allows to change the studied dataset 

In [ ]:
# Index of the file to study:
IND_FILE = 0

params_list = ["study2.json", #0
               "study4_CF_FC.json", #1
              ]

path_data = "./data/Blanchard/"

params_cnn = Params("src/model/parameters/params_cnn.json")
params_auto_cnn = Params("src/model/parameters/params_autoencoder_cnn.json")
params_file = Params(path_data + "parameters/" + params_list[IND_FILE])

print("TRAINING : ", params_file.csv_train)
print("TEST : ", params_file.csv_valid)
print("ROOM SIZE : ", params_file.room_size)
print("PADDING : ", params_file.padding)
print("NO CHOICE OPTION : ", params_file.no_choice)
print("PAIRS OF SEATS : ", params_file.is_couple)

#### Data Loading

In [ ]:
### TRAINING SET :
try:
    trainloader_mat = torch.load(path_data+"dataloader/"+params_file.dataloader_train)
except FileNotFoundError:
    print("Creating the Dataloader for train file...")
    trainloader_mat = load_data_loader(path = path_data + params_file.csv_train, 
                                       file_params = params_file,
                                       model_params = params_cnn,
                                       shuffle = True,
                                       augmentation = True,
                                       verbose = True)
    torch.save(trainloader_mat, path_data+"dataloader/"+params_file.dataloader_train)

### VALIDATION SET :
try:
    validloader_mat = torch.load(path_data+"dataloader/"+params_file.dataloader_valid)
except FileNotFoundError:
    print("Creating the Dataloader for test file...")
    validloader_mat = load_data_loader(path = path_data + params_file.csv_valid, 
                                       file_params = params_file,
                                       model_params = params_cnn,
                                       shuffle = False,
                                       augmentation = False,
                                       verbose = True)

    torch.save(validloader_mat, path_data+"dataloader/"+params_file.dataloader_valid)

In [ ]:
print(len(trainloader_mat.dataset))
print(len(validloader_mat.dataset))

### Training

In the following cell, comment/uncomment CNN or AutoencoderCNN to choose between CNN and CDNN training.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = CNN(room_size = params_file.room_size, 
            params = params_cnn)
# model = AutoencoderCNN(room_size = params_file.room_size, 
#                        params = params_auto_cnn)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=params_cnn.lr_opt)

loss_fn = nn.NLLLoss()

def top_1_acc(outputs, labels):
    return torch_top_n_accuracy(outputs, labels, N=1)

def top_3_acc(outputs, labels):
    return torch_top_n_accuracy(outputs, labels, N=3)

def top_5_acc(outputs, labels):
    return torch_top_n_accuracy(outputs, labels, N=5)

def top_10_acc(outputs, labels):
    return torch_top_n_accuracy(outputs, labels, N=10)


eval_fns = [top_1_acc, top_5_acc, top_10_acc]

train_test_model = TrainTest(model, trainloader_mat, validloader_mat, optimizer, loss_fn, eval_fns)

history_cnn = train_test_model.train(patience=20, max_it=200)


### Evaluation

In [ ]:
eval_fns = [top_1_acc, top_5_acc, top_10_acc, torch_weighted_l1]
train_test_model.eval_fns = eval_fns

accuracies = train_test_model.evaluate()

print("Performance for valid set (in %): ", accuracies[1])

if params_file.is_couple:
    
    accuracies = train_test_model.evaluate_couple()
    print("Performance for valid set (in %) with the new method: ", accuracies[1])